In [ ]:
!pip install spacy
!pip install sqlglot
!pip install sqlparse
!pip install accelerate
!pip install -U pandas sqlalchemy
!pip install -U sentence-transformers
!pip install -i https://pypi.org/simple/ bitsandbytes
!python -m spacy download en_core_web_md

### Get CSV data from Kaggle

We are using the [Young People Survey Dataset](https://www.kaggle.com/datasets/miroslavsabo/young-people-survey) from Kaggle. The primary reason for choosing this dataset is because it has a pretty large number of columns.

In [4]:
import os
import numpy as np
import pandas as pd

In [5]:
os.listdir('data/kaggle-young-people-survey-dataset')

['columns.csv', 'responses.csv']

In [3]:
df = pd.read_csv('data/kaggle-young-people-survey-dataset/responses.csv')
df

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,22.0,172.0,59.0,1.0,female,right handed,college/bachelor degree,yes,city,house/bungalow
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,female,right handed,secondary school,no,village,house/bungalow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,5.0,2.0,5.0,2.0,2.0,5.0,4.0,4.0,4.0,3.0,...,20.0,164.0,57.0,1.0,female,right handed,secondary school,no,city,house/bungalow
1006,4.0,4.0,5.0,1.0,3.0,4.0,1.0,4.0,1.0,1.0,...,27.0,183.0,80.0,5.0,male,left handed,masters degree,no,village,house/bungalow
1007,4.0,3.0,1.0,1.0,2.0,2.0,2.0,3.0,4.0,1.0,...,18.0,173.0,75.0,0.0,female,right handed,secondary school,yes,city,block of flats
1008,5.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,4.0,1.0,...,25.0,173.0,58.0,1.0,female,right handed,college/bachelor degree,no,city,block of flats


In [6]:
table_name = 'young-people-survey'

### Load data into SQLite DB

For a demonstration of real-world usecases, we will load our CSV data into a database. Once done, we will extract the DDL or the Database Definition Language, i.e. the commands which actually define our table, or more accurately, our table schema.

We will use `sqlite` for this usecase but you can use your DB of choice. You will need to of course install any relevant libraries to communicate with your DB using python.
For simplicity's sake, we use built-in functions of Pandas to load and read data from the database. Of course, you can use your DB connector as well to peform these operations.

In [2]:
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///mysqlitedb.db")

In [6]:
try:
    df.to_sql(table_name, engine, index=False)
except Exception as e:
    print(f'Error: {e}')

`sqlite` databases contain a table known as `sqlite_master` which contains the details of all tables hosted by the database. This table contains the "derived" DDL for each table. If you are using any other database, the process for getting the DDL for your table might be different.

In [7]:
with engine.connect() as conn, conn.begin():
    sqlite_master = pd.read_sql_query("SELECT * FROM sqlite_master", conn)
sqlite_master['sql_fmt'] = sqlite_master['sql'].apply(lambda z: [x.strip().strip(',').rsplit(' ', maxsplit=1) for x in z.split('\n')[1:-1]])

table_desc_dict = {}
for _, row in sqlite_master.iterrows():
    table_desc_dict[row['name']] = row['sql']

schema = table_desc_dict[table_name]
print(schema)

CREATE TABLE "young-people-survey" (
	"Music" FLOAT, 
	"Slow songs or fast songs" FLOAT, 
	"Dance" FLOAT, 
	"Folk" FLOAT, 
	"Country" FLOAT, 
	"Classical music" FLOAT, 
	"Musical" FLOAT, 
	"Pop" FLOAT, 
	"Rock" FLOAT, 
	"Metal or Hardrock" FLOAT, 
	"Punk" FLOAT, 
	"Hiphop, Rap" FLOAT, 
	"Reggae, Ska" FLOAT, 
	"Swing, Jazz" FLOAT, 
	"Rock n roll" FLOAT, 
	"Alternative" FLOAT, 
	"Latino" FLOAT, 
	"Techno, Trance" FLOAT, 
	"Opera" FLOAT, 
	"Movies" FLOAT, 
	"Horror" FLOAT, 
	"Thriller" FLOAT, 
	"Comedy" FLOAT, 
	"Romantic" FLOAT, 
	"Sci-fi" FLOAT, 
	"War" FLOAT, 
	"Fantasy/Fairy tales" FLOAT, 
	"Animated" FLOAT, 
	"Documentary" FLOAT, 
	"Western" FLOAT, 
	"Action" FLOAT, 
	"History" FLOAT, 
	"Psychology" FLOAT, 
	"Politics" FLOAT, 
	"Mathematics" FLOAT, 
	"Physics" FLOAT, 
	"Internet" FLOAT, 
	"PC" FLOAT, 
	"Economy Management" FLOAT, 
	"Biology" FLOAT, 
	"Chemistry" FLOAT, 
	"Reading" FLOAT, 
	"Geography" FLOAT, 
	"Foreign languages" FLOAT, 
	"Medicine" FLOAT, 
	"Law" FLOAT, 
	"Cars" FLO

In [8]:
table_desc_df = {}
for _, row in sqlite_master.iterrows():
    table_desc_df[row['name']] = pd.DataFrame(columns=['name', 'type'], data=row['sql_fmt'])
    table_desc_df[row['name']]['comment'] = np.nan

table_desc_df['young-people-survey']

,name,type,comment
0,"""Music""",FLOAT,NaN
1,"""Slow songs or fast songs""",FLOAT,NaN
2,"""Dance""",FLOAT,NaN
3,"""Folk""",FLOAT,NaN
4,"""Country""",FLOAT,NaN
...,...,...,...
145,"""Left - right handed""",TEXT,NaN
146,"""Education""",TEXT,NaN
147,"""Only child""",TEXT,NaN
148,"""Village - town""",TEXT,NaN


### Prompt LLM to generate SQL
You can use your LLM of choice to generate SQL commands, but models specifically fine-tuned to generate high-quality SQL are obviously prefered. We will use the model [`llama-3-sqlcoder-8b` by Defog](https://huggingface.co/defog/llama-3-sqlcoder-8b), which was derived by finetunig the `llama-3` model released by Meta.

While we will be prompting the model as described by the model's creators, feel free to experiment with other models and other prompt templates. Other popular LLMs for code generation are [`Deepseek-Coder-V2` by DeepSeek (深度求索)](https://huggingface.co/deepseek-ai/DeepSeek-Coder-V2-Instruct) and [`CodeQwen1.5-7B-Chat` by Qwen](https://huggingface.co/Qwen/CodeQwen1.5-7B-Chat)

### Benchmarks
<details>
<summary> HumanEval </summary>

[HumanEval](https://paperswithcode.com/dataset/humaneval) problem solving dataset is introduced in the paper "Evaluating Large Language Models Trained on Code". It used to measure functional correctness for synthesizing programs from docstrings. It consists of 164 original programming problems, assessing language comprehension, algorithms, and simple mathematics, with some comparable to simple software interview questions.

</details>

<details>
<summary> EvalPlus </summary>

[EvalPlus](https://github.com/evalplus/evalplus) is a rigorous evaluation framework for LLM4Code, with:  
✨ HumanEval+: 80x more tests than the original HumanEval!  
✨ MBPP+: 35x more tests than the original MBPP!  
[...]  
Why EvalPlus? What does using EvalPlus datasets bring to you?  
✨ Reliable ranking: See our leaderboard for the latest LLM ranking before and after rigorous evaluation.  
✨ Code robustness: Look at the score differences! esp. before (e.g., HumanEval) and after (e.g., HumanEval+) using EvalPlus! The drop/gap indicates if the LLM can generate more robust code: less drop means more robustness and a larger drop means the code tends to be more fragile.
</details>

<details>
<summary> InfiCoder-eval </summary>

[InfiCoder-eval](https://infi-coder.github.io/inficoder-eval) is a large-scale free-form question-answering (QA) benchmark for code. InfiCoder-Eval comprises 270 carefully picked high-quality StackOverflow questions, covering 18 programming languages. [...] As confirmed with human experiments, InfiCoder-Eval evaluation aligns with humans better than model-based evaluation and runs much faster at the same time. [...] Existing benchmarks weigh heavily on code generation, unit-test-based evaluation, and a limited set of programming languages. InfiCoder-Eval processes a much higher diversity to reflect real-world code LLMs’ usage scenarios and is far from saturation.
</details>

<details>
<summary> LMSYS Chatbot Arena Leaderboard </summary>

[LMSYS Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard) is a crowdsourced open platform for LLM evals. We've collected over 200,000 human preference votes to rank LLMs with the Elo ranking system.
</details>

<details>
<summary> Big Code Models Leaderboard </summary>

[Big Code Models Leaderboard (open source models only)](https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard) compares performance of base multilingual code generation models on HumanEval benchmark and MultiPL-E. We also measure throughput and provide information about the models. We only compare open pre-trained multilingual code models, that people can start from as base models for their trainings. Win Rate represents how often a model outperforms other models in each language, averaged across all languages. The scores of instruction-tuned models might be significantly higher on humaneval-python than other languages. [...] HumanEval-Python reports the pass@1 on HumanEval; the rest is from MultiPL-E benchmark. For all languages, we use the original benchamrk prompts for all models except HumanEval-Python, where we separate base from instruction models.
</details>

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "defog/llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# else, load in 8 bits – this is a bit slower
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    # torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
    use_cache=True,
)

In [ ]:
import sqlparse

def generate_query(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

Finetuned LLMs are very sensitive to the prompt. Make sure you are using the prompt format specified by the creators of the model. You can usually get this information in the HuggingFace model card or in Github repos.

In [15]:
question = 'Which are the rows which have 1 for at least one music related column?'

prompt_template = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION] 

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

## Database Schema 
This query will run on a database whose schema is represented in this string: {db_schema} 

### Answer 
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION] [SQL]
"""

prompt = prompt_template.format(question=question, db_schema=schema)
print(prompt)

### Task
Generate a SQL query to answer [QUESTION]Which are the rows which have 1 for at least one music related column?[/QUESTION] 

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

## Database Schema 
This query will run on a database whose schema is represented in this string: CREATE TABLE "young-people-survey" (
	"Music" FLOAT, 
	"Slow songs or fast songs" FLOAT, 
	"Dance" FLOAT, 
	"Folk" FLOAT, 
	"Country" FLOAT, 
	"Classical music" FLOAT, 
	"Musical" FLOAT, 
	"Pop" FLOAT, 
	"Rock" FLOAT, 
	"Metal or Hardrock" FLOAT, 
	"Punk" FLOAT, 
	"Hiphop, Rap" FLOAT, 
	"Reggae, Ska" FLOAT, 
	"Swing, Jazz" FLOAT, 
	"Rock n roll" FLOAT, 
	"Alternative" FLOAT, 
	"Latino" FLOAT, 
	"Techno, Trance" FLOAT, 
	"Opera" FLOAT, 
	"Movies" FLOAT, 
	"Horror" FLOAT, 
	"Thriller" FLOAT, 
	"Comedy" FLOAT, 
	"Romantic" FLOAT, 
	"Sci-fi" FLOAT, 
	"War" FLOAT, 
	"Fantasy/Fairy tales" FLOAT, 
	"Animated" FLOAT, 
	"Documentary" FLOAT, 
	"Western" FLO

In [ ]:
generated_sql = generate_query(prompt)
print(f'SQL generated by the model: {generated_sql}')

Finally we execute the generated SQL query on our DB and verify its performance.

In [ ]:
engine = create_engine(f"sqlite:///mysqlitedb.db")

with engine.connect() as conn, conn.begin():
    query_result = pd.read_sql_query(generated_sql, conn)

query_result